## 0 - Bibliotecas

In [184]:
#Para scraping
from bs4 import BeautifulSoup
import requests

#Para manipulação de dados
import pandas as pd
import numpy as np




## 1 - Request da página


In [185]:
url = "https://lol.fandom.com/wiki/CBLOL/2023_Season/Split_2/Player_Statistics" #url da página escolhida para scraping

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [186]:
#Código html da página desejada
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   CBLOL 2023 Split 2 - Player Statistics - Leaguepedia | League of Legends Esports Wiki
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ddadd24f66f38321d459169585f9e6d1","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"CBLOL/2023_Season/Split_2/Player_Statistics","wgTitle":"CBLOL/2023 Season/Split 2/Player Statistics","wgCurRevisionId":3551633,"wgRevisionId":3551633,"wgArticleId":801327,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Tournament Player Statisti

## 2 - Seleção da tabela de interesse

Usando o método find_all encontramos todas as tags 'table', dentre elas selecionamos a que contem os dados que procuramos.

In [187]:
#Tabela desejada
table_stats_cblol = soup.find_all('table')[4]
table_stats_cblol

<table class="wikitable sortable spstats plainlinks hoverable-rows"><tbody><tr><th class="colspan-cell" colspan="21">Tournament: <a class="to_hasTooltip" data-to-flags="fieM" data-to-id="CBLOL_2f-2023_Season_2f-Split_2" data-to-titles="CBLOL/2023 Season/Split 2||Tooltip:CBLOL/2023 Season/Split 2" href="/wiki/CBLOL/2023_Season/Split_2" title="CBLOL/2023 Season/Split 2">CBLOL/2023 Season/Split 2</a> - Showing Values Per Game - <a class="text" href="https://lol.fandom.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=CBLOL/2023%20Season/Split%202&amp;TS%5Bpreload%5D=TournamentByPlayer&amp;_run=" rel="noreferrer noopener" target="_blank">Open As Query</a></th></tr><tr><th colspan="21">Player Statistics - 90 Total Games Played, 60 Players</th></tr><tr><th class="nested-table-outer" colspan="21"><table class="nested-table wikitable"><tbody><tr><th class="spstats-rolelinks-title">Restrict By Role (Query Permalinks):</th><td class="spstats-rolelinks-cell"><a class="text" href="https

Aqui busco encontrar os rótulos referentes às colunas da tabela com o método find_all, após inspecionar a página da qual estou extraindo os dados, com o obejtivo de criar a estrutura do dataframe.

In [188]:
#Nomes das colunas
nomes_colunas = table_stats_cblol.find_all('th')
nomes_colunas

[<th class="colspan-cell" colspan="21">Tournament: <a class="to_hasTooltip" data-to-flags="fieM" data-to-id="CBLOL_2f-2023_Season_2f-Split_2" data-to-titles="CBLOL/2023 Season/Split 2||Tooltip:CBLOL/2023 Season/Split 2" href="/wiki/CBLOL/2023_Season/Split_2" title="CBLOL/2023 Season/Split 2">CBLOL/2023 Season/Split 2</a> - Showing Values Per Game - <a class="text" href="https://lol.fandom.com/Special:RunQuery/TournamentStatistics?TS%5Btournament%5D=CBLOL/2023%20Season/Split%202&amp;TS%5Bpreload%5D=TournamentByPlayer&amp;_run=" rel="noreferrer noopener" target="_blank">Open As Query</a></th>,
 <th colspan="21">Player Statistics - 90 Total Games Played, 60 Players</th>,
 <th class="nested-table-outer" colspan="21"><table class="nested-table wikitable"><tbody><tr><th class="spstats-rolelinks-title">Restrict By Role (Query Permalinks):</th><td class="spstats-rolelinks-cell"><a class="text" href="https://lol.fandom.com/Special:RunQuery/TournamentStatistics?TS%5Bpreload%5D=TournamentByPlayer

In [189]:
#Nomes das colunas da tabela
titulos_colunas = [nomes.text for nomes in nomes_colunas][4:]
titulos_colunas

['',
 '',
 'G',
 'W',
 'L',
 'WR',
 'K',
 'D',
 'A',
 'KDA',
 'CS',
 'CS/M',
 'G',
 'G/M',
 'DMG',
 'DMG/M',
 'KPAR',
 'KS',
 'GS',
 'CP',
 'Champs']

**Problema**: os dois primeiros valores são vazios. Preciso preencher eles com os rótulos referentes as colunas.

##3 - Criando estrutura do Dataframe

Uma vez criada a lista com os nomes referência para cada coluna da tabela, criei a estrutura do dataframe. Preciso também renomear as calunas sem nome, usei a biblioteca pandas para tal.

In [190]:
#criando o dataframe
df_cblol = pd.DataFrame(columns = titulos_colunas)

#Atribuindo títulos distintos para colunas de mesmo título
cols = []
count = 1
for column in df_cblol.columns:
    if column == '':
        cols.append(f'col_{count}')
        count+=1
        continue
    cols.append(column)
df_cblol.columns = cols

#Atribuindo os títulos finais
df_cblol.rename(columns={"col_1":"Team"}, inplace=True)
df_cblol.rename(columns={"col_2":"Player"}, inplace=True)


In [191]:
#Estrutura do dataframe criada
df_cblol

,Team,Player,G,W,L,WR,K,D,A,KDA,...,CS/M,G,G/M,DMG,DMG/M,KPAR,KS,GS,CP,Champs


In [192]:
#Procurando os dados para alimentar o dataframe
#tr é a tag que indica as linhas. r = row
column_data = table_stats_cblol.find_all('tr')[5:]
column_data

[<tr><td class="spstats-team">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Fluxo" data-to-titles="Fluxo||Tooltip:Fluxo" href="/wiki/Fluxo" title="Fluxo"><img alt="Fluxologo std" class="black-logo-std" data-file-height="25" data-file-width="60" data-image-key="Fluxologo_std.png" data-image-name="Fluxologo std.png" data-relevant="0" decoding="async" height="25" loading="lazy" src="https://static.wikia.nocookie.net/lolesports_gamepedia_en/images/3/3d/Fluxologo_std.png/revision/latest?cb=20230310092442" width="60"/></a>⁠</td><td class="spstats-player"><a class="catlink-players pWAG pWAN to_hasTooltip" data-to-flags="fiem" data-to-id="Brance" data-to-titles="Brance||Tooltip:Brance" href="/wiki/Brance" title="Brance">Brance</a></td><td><a class="text" href="https://lol.fandom.com/Special:RunQuery/MatchHistoryPlayer?MHP%5Bpreload%5D=Player&amp;MHP%5Btournament%5D=CBLOL/2023%20Season/Split%202&amp;MHP%5Blink%5D=Brance&amp;_run=" rel="noreferrer noopener" target="_blank">18</a></t

In [193]:
#Adicionando dados para cada linha do dataframe
#td é a tag que indica os dados. d = data.
for row in column_data:
  row_data = row.find_all('td')
  data_row_individual = [data.text for data in row_data]
  lenth = len(df_cblol)
  df_cblol.loc[lenth] = data_row_individual


## 4 - Colunas com nome e dados faltantes.

Até aqui preenchi praticamente todo o dataframe de uma única vez, porém as colunas 'Team' e 'Champ' estão vazias.

As tags necessárias para acessar os dados da coluna 'Team' não são as mesmas  para as demais colunas. Em virtude disso, fiz uma nova solicitação em busca das tags corretas.

In [194]:
teams = soup.find_all(['td'], class_ = ['spstats-team'])
teams

[<td class="spstats-team">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Fluxo" data-to-titles="Fluxo||Tooltip:Fluxo" href="/wiki/Fluxo" title="Fluxo"><img alt="Fluxologo std" class="black-logo-std" data-file-height="25" data-file-width="60" data-image-key="Fluxologo_std.png" data-image-name="Fluxologo std.png" data-relevant="0" decoding="async" height="25" loading="lazy" src="https://static.wikia.nocookie.net/lolesports_gamepedia_en/images/3/3d/Fluxologo_std.png/revision/latest?cb=20230310092442" width="60"/></a>⁠</td>,
 <td class="spstats-team">⁠<a class="to_hasTooltip" data-to-flags="fiem" data-to-id="Fluxo" data-to-titles="Fluxo||Tooltip:Fluxo" href="/wiki/Fluxo" title="Fluxo"><img alt="Fluxologo std" class="black-logo-std" data-file-height="25" data-file-width="60" data-image-key="Fluxologo_std.png" data-image-name="Fluxologo std.png" data-relevant="0" decoding="async" height="25" loading="lazy" src="https://static.wikia.nocookie.net/lolesports_gamepedia_en/images/3/3d

In [195]:
#Adicionando os valores à coluna 'Team' e limpandos os referentes dados.
names_team = []
for team in teams:
  row_team = team.find('a')
  names_team.append(row_team.get('data-to-id'))

names_cblol_team = pd.Series(names_team)
names_cblol_team.replace({'KaBuM_21-_Esports':'KaBuM! Esports',
                          'Los_Grandes':'Los Grandes',
                          'PaiN_Gaming':'PaiN Gaming',
                          'RED_Canids':'RED Canids',
                          'Vivo_Keyd_Stars':'Vivo Keyd Stars'}, inplace=True)


In [196]:
#Verificando os dados
names_cblol_team

0               Fluxo
1               Fluxo
2               Fluxo
3               Fluxo
4               Fluxo
5               FURIA
6               FURIA
7               FURIA
8               FURIA
9               FURIA
10               INTZ
11               INTZ
12               INTZ
13               INTZ
14               INTZ
15     KaBuM! Esports
16     KaBuM! Esports
17     KaBuM! Esports
18     KaBuM! Esports
19     KaBuM! Esports
20     KaBuM! Esports
21     KaBuM! Esports
22            Liberty
23            Liberty
24            Liberty
25            Liberty
26            Liberty
27            Liberty
28            Liberty
29            Liberty
30            Liberty
31            Liberty
32            Liberty
33        Los Grandes
34        Los Grandes
35        Los Grandes
36        Los Grandes
37        Los Grandes
38        Los Grandes
39        Los Grandes
40               LOUD
41               LOUD
42               LOUD
43               LOUD
44               LOUD
45        

In [197]:
#Atribuindo os valores à coluna 'Team' no dataframe
df_cblol['Team'] = names_cblol_team


In [198]:
df_cblol

,Team,Player,G,W,L,WR,K,D,A,KDA,...,CS/M,G,G/M,DMG,DMG/M,KPAR,KS,GS,CP,Champs
0,Fluxo,Brance,18,11,7,61.1%,6.33,2.11,4.67,5.21,...,9.91,16.1,473,29.9k,876.5,78.6%,45.2%,26.1%,7,
1,Fluxo,Disamis,18,11,7,61.1%,2.22,2.11,7.94,4.82,...,5.13,11.2,329,11.4k,335.5,72.6%,15.9%,18.2%,9,
2,Fluxo,hauz,18,11,7,61.1%,3.06,2.78,6.06,3.28,...,8.3,13.7,402,22.4k,658.5,65.1%,21.8%,22.2%,7,
3,Fluxo,JoJo,18,11,7,61.1%,0.39,2.61,10.83,4.3,...,1.13,8.1,237,5.2k,153.7,80.2%,2.8%,13.1%,6,
4,Fluxo,Tay,18,11,7,61.1%,2,1.89,6.11,4.29,...,7.46,12.6,370,12.8k,374.5,57.9%,14.3%,20.4%,8,
5,FURIA,Ayu,18,8,10,44.4%,3.17,2.28,5.56,3.83,...,9.38,14.7,430,19.9k,581.5,71%,25.8%,24.4%,11,
6,FURIA,Betao,18,8,10,44.4%,1.72,2.22,4.83,2.95,...,7.39,12.3,358,16.4k,479.1,53.4%,14%,20.3%,8,
7,FURIA,Goot,18,8,10,44.4%,2.78,2.56,6.33,3.57,...,5.21,11.4,335,11k,322.5,74.2%,22.6%,19%,9,
8,FURIA,Tutsz,18,8,10,44.4%,3.11,2.22,6,4.1,...,8.4,13.8,403,20.3k,594.3,74.2%,25.3%,22.8%,10,
9,FURIA,zay,18,8,10,44.4%,1.5,2.44,8.11,3.93,...,0.99,8.2,239,5.7k,167.3,78.3%,12.2%,13.5%,10,



## 4.1 - Configurando os tipos de dados
**Dtypes das colunas do dataframe**

Nem todos os dados até o momento têm o tipo mais desejado, essa seção voltamos nossa atenção a isso.

'Team' = str

 'Player' = str

 'G' = int

 'W' = int

 'L' = int

 'WR' = obj

 'K' = float

 'D' = float

 'A' = float

 'KDA' = float

 'CS' = float

 'CS/M' = float

 'G' = obj

 'G/M' = float

 'DMG'= obj

 'DMG/M' = float

 'KPAR' = obj

 'KS' = obj

 'GS' = obj

 'CP' = int

 'Champs' = str

In [199]:
#dtype atuais
df_cblol.dtypes

Team      object
Player    object
G         object
W         object
L         object
WR        object
K         object
D         object
A         object
KDA       object
CS        object
CS/M      object
G         object
G/M       object
DMG       object
DMG/M     object
KPAR      object
KS        object
GS        object
CP        object
Champs    object
dtype: object

In [200]:
#Estabelecendo os tipos de dados corretos
df_cblol = df_cblol.astype({'Team': str,
                  'Player': str,
                  'G': int,
                  'W': int,
                  'L': int,
                  'WR': object,
                  'K': float,
                  'D': float,
                  'A': float,
                  'KDA': float,
                  'CS': float,
                  'CS/M': float,
                  'G': object,
                  'G/M': float,
                  'DMG': object,
                  'DMG/M': float,
                  'KPAR': object,
                  'KS': object,
                  'GS': object,
                  'CP': int,
                  'Champs': str})

df_cblol['G'] = (df_cblol['G'].astype(float)).astype(int) #a feature G não mudou o dtype na primeira reconfiguração

In [201]:
#dtype atuais corrigidos
df_cblol.dtypes

Team       object
Player     object
G           int64
W           int64
L           int64
WR         object
K         float64
D         float64
A         float64
KDA       float64
CS        float64
CS/M      float64
G           int64
G/M       float64
DMG        object
DMG/M     float64
KPAR       object
KS         object
GS         object
CP          int64
Champs     object
dtype: object

## 4.2 - Tratamento da coluna champs

**Problema** A coluna 'Champs' refere-se, originalmente, a listas com três valores cada referentes a cada jogador. Optei por mudar a coluna de 'Champs' para 'MPC'(most played champion), indicando não campeões mais jogados (três para cada), mas sim O campeão mais jogado (um para cada) por jogador.




In [202]:
#lista com informações dos champions contidas na tag span
table_played_champion = soup.find_all('span', class_ = 'sprite champion-sprite')[10:]
table_played_champion
# table_played_champion = soup.find_all('a', class_ = 'text')
# table_played_champion



[<span class="sprite champion-sprite" style="background-position:-78px -78px;background-size:387px auto;width:25px;height:25px" title="Kai'Sa"></span>,
 <span class="sprite champion-sprite" style="background-position:-0px -258px;background-size:387px auto;width:25px;height:25px" title="Aphelios"></span>,
 <span class="sprite champion-sprite" style="background-position:-284px -258px;background-size:387px auto;width:25px;height:25px" title="Zeri"></span>,
 <span class="sprite champion-sprite" style="background-position:-258px -103px;background-size:387px auto;width:25px;height:25px" title="Maokai"></span>,
 <span class="sprite champion-sprite" style="background-position:-310px -207px;background-size:387px auto;width:25px;height:25px" title="Wukong"></span>,
 <span class="sprite champion-sprite" style="background-position:-181px -258px;background-size:387px auto;width:25px;height:25px" title="Viego"></span>,
 <span class="sprite champion-sprite" style="background-position:-52px -0px;backg

**Problema**: Para o caso de jogadores inscritos pós abertura do campeonato e, consequentemente, acumulam menos de 3 campeões jogados, os valores tornam-se inconsistentes com os respectivos jogadores.

In [203]:
#Indices das ocorrências de jogadores com menos de 3 jogos na lista table_played_champion.
print(list(np.where(df_cblol['CP']<3)))

[array([21, 32])]


In [204]:
mpc_cblol = [] #três campeões mais jogados por cada jogador
for champion in table_played_champion:
  mpc_cblol.append(champion.get('title'))

mpc = [] #campeão mais jogado por cada jogador

n = range(0, len(mpc_cblol), 3)
for i in n:
   mpc.append(mpc_cblol[i])
   if len(mpc) == 22:
    break

n1 = range(64, len(mpc_cblol), 3) #(1º indice problemático x 3) + 1, indice em table_played_champion 64.
for j in n1:
  mpc.append(mpc_cblol[j])
  if len(mpc) == 33:
    break

n2 = range(95, len(mpc_cblol), 3) #(2º indice problemático x 3) + 1, indice em table_played_champion 95.
for k in n2:
  mpc.append(mpc_cblol[k])


mpc = pd.Series(mpc) #armazenando como uma serie
mpc





0        Kai'Sa
1        Maokai
2          Ahri
3         Rakan
4       K'Sante
5        Kai'Sa
6         Poppy
7       Sejuani
8         Annie
9          Pyke
10        Annie
11         Rell
12        Xayah
13           Vi
14         Olaf
15        Jayce
16     Renekton
17       Kai'Sa
18        Rakan
19    Jarvan IV
20        Poppy
21      Sejuani
22      Sejuani
23         Ahri
24     Aphelios
25      K'Sante
26        Rakan
27         Rell
28       Maokai
29         Ahri
30       Kai'Sa
31     Renekton
32      Alistar
33         Ahri
34     Aphelios
35         Rell
36      Sejuani
37         Ornn
38          Jax
39      Sejuani
40        Rakan
41        Poppy
42     Renekton
43        Xayah
44       Syndra
45     Aphelios
46      Sejuani
47         Ahri
48     Nautilus
49         Gnar
50       Wukong
51        Annie
52      K'Sante
53        Rakan
54     Aphelios
55        Rakan
56       Wukong
57        Annie
58         Gwen
59       Kai'Sa
dtype: object

Fracionar o append para a lista mpc solucionou provisoriamente o problema.

In [205]:
#modificando a última coluna e adicionando valores
df_cblol.rename(columns = {'Champs':'mpc'}, inplace = True)
df_cblol['mpc'] = mpc




## 5 - Adicionando coluna "Role"

É interessante também poder analisar os jogadores por suas respectivas funções no jogo A feature "Role" tem a função de informar a posição de cada jogador dentro do seu time.

In [206]:
players = df_cblol['Player']
players

0        Brance
1       Disamis
2          hauz
3          JoJo
4           Tay
5           Ayu
6         Betao
7          Goot
8         Tutsz
9           zay
10     NOsFerus
11          Nia
12    NinjaKiwi
13        Yampi
14          Zzk
15     Krastyel
16       Lonely
17        RAV3N
18      RedBert
19     Flawless
20        SCARY
21        Samkz
22        accez
23      Avenger
24      Juliera
25        Kiari
26          Wos
27       Cavalo
28    Drakehero
29       Piloto
30       Reaper
31          SkB
32      Momochi
33         Lava
34       Netuno
35       Ranger
36        enel1
37        Hidan
38         Boal
39        Trick
40         Ceos
41         Croc
42         Robo
43        Route
44      tinowns
45         Bvoy
46       CarioK
47     dyNquedo
48     ProDelta
49        Wizer
50        Aegis
51         Envy
52          fNb
53      frostyy
54        TitaN
55       Damage
56        Grell
57     Grevthar
58        Guigo
59        Trigo
Name: Player, dtype: object

In [226]:
role = ['Top', 'Jungle', 'Mid', 'Bot', 'Sup'] #rótulos

#Função que atribui a cada jogador um rótulo
def role_player(players):
  if players in ["fNb", "Wizer","Robo", "Tay", "Zzk", "Guigo", "Betao",
                 "Lonely", "Kiari", "Hidan", "Boal", "SkB"]:
    return role[0]

  elif players in ["Aegis", "CarioK", "Croc", "Disamis", "Yampi", "Grell",
                   "Goot", "enel1", "accez", "Flawless", "SCARY",
                   "Trick", "Drakehero", "Samkz"]:
    return role[1]

  elif players in ["dyNquedo","Envy","tinowns","hauz","NOsFerus",
                   "Grevthar","Tutsz","Lava","Krastyel","Avenger","Piloto"]:
    return role[2]

  elif players in ["Route","Brance","NinjaKiwi","Trigo","Ayu",
                   "Netuno","RAV3N","Juliera","Reaper"]:
    return role[3]
  else:
  # elif players in ["frostyy","ProDelta","Ceos","JoJo","Nia","Damage","zay","Ranger","RedBert","Wos","Cavalo","Momochi"]:
    return role[4]


In [227]:
#Criando a feature 'Role'
df_cblol["Role"] = df_cblol["Player"].apply(role_player)


In [228]:
df_cblol

,Team,Player,G,W,L,WR,K,D,A,KDA,...,G,G/M,DMG,DMG/M,KPAR,KS,GS,CP,mpc,Role
0,Fluxo,Brance,18,11,7,61.1%,6.33,2.11,4.67,5.21,...,16,473.0,29.9k,876.5,78.6%,45.2%,26.1%,7,Kai'Sa,Bot
1,Fluxo,Disamis,18,11,7,61.1%,2.22,2.11,7.94,4.82,...,11,329.0,11.4k,335.5,72.6%,15.9%,18.2%,9,Maokai,Jungle
2,Fluxo,hauz,18,11,7,61.1%,3.06,2.78,6.06,3.28,...,13,402.0,22.4k,658.5,65.1%,21.8%,22.2%,7,Ahri,Mid
3,Fluxo,JoJo,18,11,7,61.1%,0.39,2.61,10.83,4.30,...,8,237.0,5.2k,153.7,80.2%,2.8%,13.1%,6,Rakan,Sup
4,Fluxo,Tay,18,11,7,61.1%,2.00,1.89,6.11,4.29,...,12,370.0,12.8k,374.5,57.9%,14.3%,20.4%,8,K'Sante,Top
5,FURIA,Ayu,18,8,10,44.4%,3.17,2.28,5.56,3.83,...,14,430.0,19.9k,581.5,71%,25.8%,24.4%,11,Kai'Sa,Bot
6,FURIA,Betao,18,8,10,44.4%,1.72,2.22,4.83,2.95,...,12,358.0,16.4k,479.1,53.4%,14%,20.3%,8,Poppy,Top
7,FURIA,Goot,18,8,10,44.4%,2.78,2.56,6.33,3.57,...,11,335.0,11k,322.5,74.2%,22.6%,19%,9,Sejuani,Jungle
8,FURIA,Tutsz,18,8,10,44.4%,3.11,2.22,6.00,4.10,...,13,403.0,20.3k,594.3,74.2%,25.3%,22.8%,10,Annie,Mid
9,FURIA,zay,18,8,10,44.4%,1.50,2.44,8.11,3.93,...,8,239.0,5.7k,167.3,78.3%,12.2%,13.5%,10,Pyke,Sup


## 6 - Salvando o DataFrame como um csv

Agora que finalizamos todo o scraping e criamos nosso dataframe, podemos salva-lo facilmente para exportação.

In [211]:
#Salvando como csv
df_cblol.to_csv('cblol_players_stats.csv', sep = ',', index=False)

##Análise exploratória

